### Previous on Weekly-Lab
上次动手实验室，
我们完成了一个爬虫，
获取了校园卡的消费记录。

无论上次你参没参加，
无论你完成得好与不好，
为了统一进度，
还请同步git仓库。

### 依赖

- requests
- pandas
- BeautifulSoup
- matplotlib
- Pillow

安装
- windows  `py -3 -m pip install beautifulsoup4 matplotlib pandas Pillow requests` 
- unix     `python3 -m pip install beautifulsoup4 matplotlib pandas Pillow requests` 

## 获取消费记录

如果你已经同步了仓库，那么你的目录下便会有一个`tutorial_ecard_2.py`文件。用Python3运行它，能获取到校园卡的消费信息。

没出意外的话，你的目录下应该已经有了`info.csv`。你可以用文本编辑器打开它，里面正是你的消费记录。

## pandas
> 最大的心愿是去掉黑眼圈和拍一张彩色照片。

pandas 是一种列存数据分析 API，它是用于处理和分析输入数据的强大工具。

先导入pandas库

In [ ]:
import pandas as pd
pd.__version__

#### pandas 主要有两种数据结构：
  - Series，它是一列数据。大概类似于 Python 中的列表。
  - DataFrame，类似于表格。DataFrame 中包含一个或多个 Series。Series 便是表格中的列，每个 Series 都有一个列名。
  

| A  | B  |  C | D  |  E  | F
---|---|---|---|---|---|---
0  | 1.0 |2013-01-02 | 1.0 | 3 |  test | foo
1  | 1.0 |2013-01-02 | 1.0 | 3 | train | foo
2  | 1.0 |2013-01-02 | 1.0 | 3 |  test | foo
3  | 1.0 |2013-01-02 | 1.0 | 3 | train | foo

上表中的
- A {1.0,1.0,1.0,1.0}
- B {2013-01-02,2013-01-02,2013-01-02,2013-01-02}
- ...
- E {test,train,test,train}
- ...

便相当于 Sereis，整个表格便相当于DataFrame。


我们可以用 `pandas.DataFrame` 和 `pandas.Series` 构造这两个结构。

In [ ]:
pd.Series(['San Francisco', 'San Jose', 'Sacramento']) # 用列表生成 Series

In [ ]:
pd.date_range('20180401', periods=6) # pandas 也有一些生成 Series 的方法

In [ ]:
city_names = pd.Series(['San Francisco', 'San Jose', 'Sacramento'])
population = pd.Series([852469, 1015785, 485199])

# 而 DataFrame 可以用 列名:Series 这样的字典创建。
cities = pd.DataFrame({ 'City name': city_names, 'Population': population }) 

cities

既然 DataFrame 可以用字典来构造，一般来说 DataFrame 也会表现得比较像字典。在很多对象中都有这样的规律。

In [ ]:
cities['City name']

In [ ]:
cities['Population']

In [ ]:
cities['Area square miles'] = pd.Series([46.87, 176.53, 97.92]) # 可以直接往里面添加/修改列

cities

同样，Series 也表现得很像列表。

In [ ]:
cities['City name'][0]

In [ ]:
cities['City name'][0:2]

#### 操纵数据
用pandas可以很简便地操纵数据。例如：

In [ ]:
population / 1000.0

并且，你可以用 .apply 函数，实现 map 的功能。

In [ ]:
population.apply(lambda val: val > 1000000) # 生成新列，布尔值表示 population 是否大于 1,000,000

甚至，你可以直接做列与列的计算

In [ ]:
cities['Population density'] = cities['Population'] / cities['Area square miles'] # 人口密度 = 人口 / 面积
cities

练习：已知San Francisco, San Jose, Sacramento 的 GDP 分别为：43170.4, 23522.2, 11882.2 （单位：百万美元）。为 DataFrame 添加 GDP 列，并计算人均 GDP (单位：美元）。（参考数据不一定真实）

In [ ]:
cities['GDP'] = 
cities['GDP per capita'] =
cities

参考答案
<p style="color:#000;background-color:#000;">
cities['GDP'] = pd.Series([43170.4, 23522.2, 11882.2])<br/>
cities['GDP per capita'] = cities['GDP'] * 1000000 / cities['Population']
</p>

另外，对 Series 和 DataFrames 还有一些统计相关的方法。你可以从[这里](http://pandas.pydata.org/pandas-docs/stable/basics.html#descriptive-statistics)查看更多的文档。

In [ ]:
cities['Population'].sum() # 计算总人口

In [ ]:
cities['Population'].mean() # 计算平均人口

In [ ]:
cities['Population'].max() # 计算最大值

## 消费数据的处理

pandas 提供了一些API供我们读取数据，常见的，用 `pandas.read_csv` 读取csv文件（这个函数不仅可以读取本地的，也可以读取网络的）。

In [ ]:
consume_data = pd.read_csv('data.csv') 
consume_data

操作时间 这一列的数据的类型是字符串，我们先转化一下格式。

In [ ]:
consume_data['操作时间'] = pd.to_datetime(consume_data['操作时间']) # 格式化“操作时间”

我们可以用 `groupby` 函数，对data进行分类。`groupby`第一个参数（也是通常使用的参数）是分类的依据，一般是列名。

In [ ]:
grouped = consume_data.groupby('科目描述') # 按照科目描述这列分类

grouped.get_group('餐费支出') # 例如这样就获得了所有餐费支出的数据

也可以对 grouped 进行诸如求和这样的操作。

In [ ]:
grouped.sum()

但是，钱包余额也被求和了，这是我们不想要的。我们可以仅选中其中的几列。

In [ ]:
grouped[['钱包交易金额']].sum()
# 为什么这里索引用的是 [[ ]] 呢？去掉里面的方括号试一下，想一下为什么。

提示:<span style="color:#000;background-color:#000;">
[] 得到的是一个 Series, [[ ]] 得到的是一个 DataFrame。
</span>

`groupby`也支持用`Grouper`对象进行分类，例如下面这段代码就按每天分类。

In [ ]:
grouped2 = consume_data.groupby(pd.Grouper(freq='1D',key='操作时间')) 
grouped2[['钱包交易金额']].sum()

练习：获得在每个终端消费的总金额。

In [ ]:
grouped3 = ______
grouped3______

参考答案：
<p style="color:#000;background-color:#000;">
grouped3 = consume_data.groupby('终端名称') <br/>
grouped3[['钱包交易金额']].sum()
</p>

## 绘制图表

在绘制图标之前，我们需要先设置一下字体，否则中文字符会无法显示。

如果设置了之后，中文仍不能正常显示，可以：
1. 安装 pypinyin
   - windows  `py -3 -m pip install pypinyin` 
   - unix     `python3 -m pip install pypinyin` 
2. 运行目录下的`topinyin.py`将所有中文字符转换成拼音

In [ ]:
%matplotlib inline 
# 上面一句用于jupyter，非jupyter请删掉。

import matplotlib
matplotlib.rc('font', family=['KaiTi']) # 在这个列表里加入字体名
matplotlib.pyplot.rcParams['axes.unicode_minus']=False # 用来正常显示负号

我们可以绘制钱包余额关于操作时间的折线图。

In [ ]:
consume_data.plot(x='操作时间',y='钱包余额')

也可以指定kind参数绘制饼状图。

In [ ]:
grouped.sum().plot(x='终端名称',y='钱包交易金额',kind='pie')

也可以绘制条形图。

In [ ]:
grouped2[['钱包交易金额']].sum().plot(kind='bar')

具体 kind 有这些选项： 
- ‘line’ : line plot (default)
- ‘bar’ : vertical bar plot
- ‘barh’ : horizontal bar plot
- ‘hist’ : histogram
- ‘box’ : boxplot
- ‘kde’ : Kernel Density Estimation plot
- ‘density’ : same as ‘kde’
- ‘area’ : area plot
- ‘pie’ : pie plot
- ‘scatter’ : scatter plot
- ‘hexbin’ : hexbin plot

练习：根据数据绘制任意一个你想要的图表